In [ ]:
for i in range(0, 4):
    print ("This is number %s" % i)


In [ ]:
result = get_backtest('536a6d181a4f090716c383b7')
result.preview('risk')

In [ ]:
aapl_minute_closes = get_pricing(
    'AAPL',
    fields='close_price', #modify to price, open_price, high, low or volume to change the field
    start_date='2014-01-01', #customize your pricing date range
    end_date = '2014-07-01',
    frequency='minute', #change to daily for daily pricing
)

# matplotlib is installed for easy plotting
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.filters import Q1500US, Q500US
from quantopian.pipeline.factors import BusinessDaysSincePreviousEvent
from quantopian.pipeline.classifiers.morningstar import Sector
import pandas as pd
import numpy as np
import math
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
aapl_minute_closes.plot()

In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.filters import Q1500US, Q500US
from quantopian.pipeline.factors import BusinessDaysSincePreviousEvent
from quantopian.pipeline.classifiers.morningstar import Sector
import pandas as pd
import numpy as np
import math
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

def make_pipeline():
    
    market_cap = Fundamentals.market_cap.latest
    total_assets = Fundamentals.total_assets.latest
    total_liability = Fundamentals.total_liabilities.latest
    net_profit = Fundamentals.gross_profit.latest
    revenue_growth = Fundamentals.revenue_growth.latest
    research_and_development = Fundamentals.research_and_development.latest
    pb_ratio = Fundamentals.pb_ratio.latest
    basic_eps = Fundamentals.basic_eps_earnings_reports.latest
    is_basic_materials = Sector().eq(101)
    is_consumer_cyclical = Sector().eq(102)
    is_financial_services = Sector().eq(103)
    is_real_estate = Sector().eq(104)
    is_consumer_defensive = Sector().eq(205)
    is_healthcare = Sector().eq(206)
    is_utilities = Sector().eq(207)
    is_communication_services = Sector().eq(308)
    is_energy = Sector().eq(309)
    is_industrials = Sector().eq(310)
    is_technology = Sector().eq(311)
    
    is_valid = basic_eps > 0 and pb_ratio > 0
       
    # Q1500US is a pre-defined universe of liquid securities. 
    # You can read more about it here: https://www.quantopian.com/posts/the-q500us-and-q1500us
    universe = Q500US() & is_valid

        #q = query(valuation.code, valuation.market_cap, balance.total_assets - balance.total_liability,
        #          balance.total_assets / balance.total_liability, income.net_profit, income.net_profit + 1, 
        #          indicator.inc_revenue_year_on_year, balance.development_expenditure).filter(
        #          valuation.code.in_(sample),
        #          indicator.eps > 0,
                  #valuation.pe_ratio > 0
        #          valuation.pb_ratio > 0
        #          )
        #df = get_fundamentals(q)    
        
    pipe = Pipeline(
              columns={
                'log_mcap': market_cap,
                'log_NC': total_assets - total_liability,
                'LEV': total_assets / total_liability,
                'NI_p': net_profit,
                'NI_n': net_profit + 1,
                'g':revenue_growth,
                'log_RD':research_and_development,
                's101': is_basic_materials,
                's102': is_consumer_cyclical,
                's103': is_financial_services,
                's104': is_real_estate,
                's205': is_consumer_defensive,
                's206': is_healthcare,
                's207': is_utilities,
                's308': is_communication_services,
                's309': is_energy,
                's310': is_industrials,
                's311': is_technology
              },
              screen = universe
          )

    return pipe

result = run_pipeline(make_pipeline(), '2018-3-15', '2018-03-15')
result.head()

In [ ]:
len(result.index)